In [1]:
from turbo import TurboM
from turbo.turbo_dkl import TurboDKL
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt

## Set up an optimization problem class

In [2]:
class Levy:
    def __init__(self, dim=10):
        self.dim = dim
        self.lb = -5 * np.ones(dim)
        self.ub = 10 * np.ones(dim)
        
    def __call__(self, x):
        assert len(x) == self.dim
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        w = 1 + (x - 1.0) / 4.0
        val = np.sin(np.pi * w[0]) ** 2 + \
            np.sum((w[1:self.dim - 1] - 1) ** 2 * (1 + 10 * np.sin(np.pi * w[1:self.dim - 1] + 1) ** 2)) + \
            (w[self.dim - 1] - 1) ** 2 * (1 + np.sin(2 * np.pi * w[self.dim - 1])**2)
        return val

f = Levy(10)

## Create a Turbo optimizer instance


In [3]:
turbo_dkl = TurboDKL(
    f=f,  # Handle to objective function
    lb=f.lb,  # Numpy array specifying lower bounds
    ub=f.ub,  # Numpy array specifying upper bounds
    n_init=10,  # Number of initial bounds from an Symmetric Latin hypercube design
    max_evals=1000,  # Maximum number of evaluations
    n_trust_regions=5,  # Number of trust regions
    batch_size=10,  # How large batch size TuRBO uses
    verbose=True,  # Print information from each batch
    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
    min_cuda=1024,  # Run on the CPU for small datasets
    device="cpu",  # "cpu" or "cuda"
    dtype="float64",  # float64 or float32
)

Using dtype = torch.float64 
Using device = cpu


# Run the optimization process

In [4]:
turbo_dkl.optimize()

TR-0 starting from: 20.71
TR-1 starting from: 9.975
TR-2 starting from: 18.36
TR-3 starting from: 33.56
TR-4 starting from: 26.94


100%|██████████| 1/1 [00:00<00:00, 54.51it/s, loss=6.06]


Updating global kernel


100%|██████████| 1/1 [00:00<00:00, 36.79it/s, loss=6.01]


60) New best @ TR-1: 9.258
Updating global kernel


...

100%|██████████| 15/15 [00:00<00:00, 26.79it/s, loss=5.02]


Updating global kernel


100%|██████████| 15/15 [00:00<00:00, 20.49it/s, loss=5.1] 


Updating global kernel


100%|██████████| 16/16 [00:00<00:00, 23.40it/s, loss=5.14]
